# [Module 1.1] 로컬 스크래치 훈련 (SageMaker 사용 안함)
 
### 본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.

- 1. 환경 셋업
- 2. 데이터 확인
- 3. 로컬 모델 훈련
- 4. 로컬 추론
- 5. 로컬에서 훈련 스크립트로 실행

## 참고:
- 세이지 메이커로 파이토치 사용 --> [Use PyTorch with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html)

---

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [ ]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('./src')

In [ ]:
import os
import numpy as np
import time

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from IPython.display import display as dp

### 커스텀 라이브러리
import config 
import model 
import evaluate 
import data_utils 

# 2. 데이터 확인
- [원본: 데이터 설명](https://github.com/hexiangnan/neural_collaborative_filtering)

## 2.1. 데이터 및 훈련 설정 파일 확인
- 사용 데이터 파일 위치 및 모델 이름 지정

In [ ]:
! pygmentize src/config.py

## 2.2. Raw 파일 확인

### train_rating
- user_id, item_id, rating, timestamp 의 4개의 컬럼으로 구성 됨.
    - 참고로 test_rating 파일을 본 코드에서 사용되지 않음

In [ ]:
! head -n5 {config.train_rating}

### test_negative
- user_id 별로 구성됨
- 아래는 첫번째 user_id "0" 의 내용 임. 
    - 첫 번째 항목은 (0,25) "0" 의 user_id 가 25 번 item_id 를 rating 했다는 것을 기록 함. 
    - 나머지 99 개는 "0" user_id 가 rating  하지 않은 item_id 99 개를 가져옴
        - 실제 모델 추론시에 top k (예; k=5)  항목을 추천을 받았을 시에  "25" 이 top k 에 포함의 유무에 따라서 performance metric 을 계산 함. 
        - 예를 들어서 추론을 하여 받은 추천 리스트가 (1064, 25, 2791, 1902, 915) 일 경우에 HR (Hit Ratio) 는 1 이 됨. 만약 (1064, 1135, 2791, 1902, 915) 처럼 25 가 없으면 HR 은 0 임.

```
(0,25)	1064	174	2791	3373	269	2678	1902	3641	1216	915	3672	2803	2344	986	3217	2824	2598	464	2340	1952	1855	1353	1547	3487	3293	1541	2414	2728	340	1421	1963	2545	972	487	3463	2727	1135	3135	128	175	2423	1974	2515	3278	3079	1527	2182	1018	2800	1830	1539	617	247	3448	1699	1420	2487	198	811	1010	1423	2840	1770	881	1913	1803	1734	3326	1617	224	3352	1869	1182	1331	336	2517	1721	3512	3656	273	1026	1991	2190	998	3386	3369	185	2822	864	2854	3067	58	2551	2333	2688	3703	1300	1924	3118
```

In [ ]:
! head -n2 {config.test_negative}

## 2.3. 훈련 및 테스트 데이터 확인
- raw 파일로 부터 훈련, 테스트 데이터 로딩
- 데이터 상세
    - 훈련 데이타는 994,169 의 rating 개수
    - 테스트 데이타는 604,000 의 rating 개수
    - user_num: 6040, item_num: 3706 의 유니크한 항목

In [ ]:
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all()




In [ ]:
import numpy as np
print("train sahpe: ", np.asarray(train_data).shape)
print("test sahpe: ", np.asarray(test_data).shape)
print(f"user_num: {user_num}, item_num: {item_num}")
print(train_data[0:5])


### 훈련 데이터 상세
- 훈련 데이타는 rating 컬럼을 사용하지 않고, user_id, item_id 두개만 사용 함.

In [ ]:
train_data_df = pd.DataFrame(train_data, columns=['user','item'])
print("train_df shape: ", train_data_df.shape)
print("train_df info: \n", train_data_df.nunique())


In [ ]:
train_data_df.sort_values(by=['user','item']).head()

user_id 당 item rating의 개수를 확인

In [ ]:
train_data_df.groupby('user').count().head()

### 테스트 데이터 상세

In [ ]:
test_data_df = pd.DataFrame(test_data, columns=['user','item'])
dp(test_data_df.head())


테스트 데이타는 user_id 당 모두 100개의 항목으로 구성 됨

In [ ]:
dp(test_data_df.groupby('user').count().head())

# 3. 로컬 모델 훈련

## 3.1. 파라미터 설정

## 모델의 하이퍼파라미터 정의
- 하아퍼 파라미터 오브젝트 이름을 args 로 생성
    - 추후 SageMaker의 Script Mode 사용사에 args 오브젝트가 사용되기에, 이름을 맞추기 위해서 같은 이름을 사용 함
- 아래 파라미터는 로직 확인 용이기에, 훈련이 빨리 끝나기 위한 파라미터 값을 설정 함(에; epoch)    
    - 약 2분 30초 소요 됨

In [ ]:
class Params:
    def __init__(self):
        self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        self.lr = 0.001
        self.top_k = 10
        self.out = True
        self.gpu = "0"
                        
args = Params()
print("# of epochs: ", args.epochs)

## 3.2. 데이터 셋 및 데이터 로더 정의

In [ ]:
train_dataset = data_utils.NCFData(
		train_data, item_num, train_mat, args.num_ng, True)

test_dataset = data_utils.NCFData(
		test_data, item_num, train_mat, 0, False)

train_loader = data.DataLoader(train_dataset,
		batch_size=args.batch_size, shuffle=True, num_workers=4)

test_loader = data.DataLoader(test_dataset,
		batch_size=args.test_num_ng+1, shuffle=False, num_workers=0)



## 3.3. 모델 네트워크 생성

In [ ]:
########################### CREATE MODEL #################################
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device: ", device)

GMF_model = None
MLP_model = None
print("Pretrained model is NOT used")    

NCF_model = model.NCF(user_num, item_num, args.factor_num, args.num_layers, 
						args.dropout, config.model, GMF_model, MLP_model)
NCF_model.to(device)


## 3.4. 손실 함수 및 옵티마이저 정의

In [ ]:
loss_function = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(NCF_model.parameters(), lr=args.lr)



## 3.5 훈련 루프 실행
- 훈련 루프에 들어가면서 `train_loader.dataset.ng_sample()` 를 통해서 Negative sample을 생성함.
    - self.num_ng = 4 * Positive Samples 만큼 생성 됨.
    - 아래는 예시 임.
```
labels_ps:  994169
labels_ng:  3976676
total train size :  4970845
```

In [ ]:
print("=====> Staring Traiing <===========")
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device: ", device)


count, best_hr = 0, 0
for epoch in range(args.epochs):
    NCF_model.train() # Enable dropout (if have).
    start_time = time.time()
    # negative sample 생성
    train_loader.dataset.ng_sample()

    for user, item, label in train_loader:
        user = user.to(device)
        item = item.to(device)
        label = label.float().to(device)

        NCF_model.zero_grad()
        prediction = NCF_model(user, item)
        loss = loss_function(prediction, label)
        loss.backward()
        optimizer.step()

        count += 1
        
    # 미자믹 배치의 user, item, label 확인
    print(f"last batch number is {count}")
    print(f"user\n{user}, item\n{item}, label\n{label}: ")


    NCF_model.eval()
    HR, NDCG = evaluate.metrics(NCF_model, test_loader, args.top_k)

    elapsed_time = time.time() - start_time
    print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
            time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
    print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

    if HR > best_hr:
        best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
        if args.out:
            if not os.path.exists(config.model_path):
                os.mkdir(config.model_path)
            torch.save(NCF_model.state_dict(),'{}{}.pth'.format(config.model_path, config.model))

            
print("End. Best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(
									best_epoch, best_hr, best_ndcg))



# 4. 로컬 추론

In [ ]:
from evaluate import predict

In [ ]:
for user, item, label in test_loader:   
    user_np = user.detach().cpu().numpy()
    item_np = item.detach().cpu().numpy()            
    break
payload = {'user':user_np.tolist(), 'item':item_np.tolist()}

print("paylaod: \n" , payload)

In [ ]:
predict(NCF_model, payload, top_k=10)

# 5. 로컬에서 훈련 스크립트로 실행

- 이번에는 훈련 코드를 스크립트 src/train_lib.py 로 만들고 train(args) 를 호출 하여 실행 합니다.
    - 위와의 차이점은 추후 SageMaker 에서 실행하기 위해서 python file 로 모듈화를 한 것 입니다.


- 중요한 단계를 로깅 합니다.
    - 훈련 환경 셋업, 
    - 데이터 준비 및 데이터 로더 생성
    - 모델 네트워크 로딩
    - 모델 훈련 시작
    - 모델 훈련 완료
    - 모델 아티펙트 저장

이번에도 하이퍼 파라미터를 정의해서 실행 합니다.

In [ ]:
class ParamsScript:
    def __init__(self):
        self.epochs = 1
        self.lr = 0.001 # 0.001 오리지널 버전        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        self.top_k = 10
        self.out = True
        self.gpu = "0"
        self.model_dir = f"{config.model_path}"                                       
        self.train_data_dir = f"{config.main_path}"               
        self.test_data_dir = f"{config.main_path}"                       

                        
script_args = ParamsScript()
print("# of epochs: ", script_args.epochs)

In [ ]:
from train_lib import train

In [ ]:
%%time 

train(script_args)

# 참고:  평가 방법

[Neural Collaborative Filtering 논문](https://arxiv.org/pdf/1708.05031.pdf)
- Evaluation Protocols. To evaluate the performance of item recommendation, we adopted the leave-one-out evalu- ation, which has been widely used in literature [1, 14, 27]. For each user, we held-out her latest interaction as the test set and utilized the remaining data for training. Since it is too time-consuming to rank all items for every user during evaluation, we followed the common strategy [6, 21] that randomly samples 100 items that are not interacted by the user, ranking the test item among the 100 items. The perfor- mance of a ranked list is judged by Hit Ratio (HR) and Nor- malized Discounted Cumulative Gain (NDCG) [11]. With- out special mention, we truncated the ranked list at 10 for both metrics. As such, the HR intuitively measures whether the test item is present on the top-10 list, and the NDCG accounts for the position of the hit by assigning higher scores to hits at top ranks. We calculated both metrics for each test user and reported the average score.